# 【百度网盘AI大赛 文档图像超分比赛】 SwinIR方案

## 一、项目背景

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;该项目基于**百度网盘AI大赛——图像处理挑战赛：文档图像超分** 比赛进行开发，下面将从比赛介绍和提交排名两个方面对项目背景进行具体阐述。

### 1.1 比赛介绍

*  **背景**：大家平时在使用手机进行拍照、扫描的时候，往往会需要将图片放大的场景，可是分辨率放大，图片中的元素都变得模糊起来，进而难以使用。本次比赛希望选手们通过算法等知识技术，帮助人们将因为放大而模糊的图片复原，提高图片分辨率，实现文档图像的“无损放大”，让其重新发挥作用。


*  **分析**：、参考文献：SwinIR: Image Restoration Using Swin Transformer（SwinIR）、本项目基于Swin IR模型进行训练。

*  **训练策略**： 采用 $128 \times 128$  的图像块进行训练。


## 二、项目方案
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;该项目采用SwinIR进行图像超分辨率，该模型的网络架构图如下:
![](https://ai-studio-static-online.cdn.bcebos.com/fb00fcd5b7524e09baa299785f6abf9483042f4bcc0e4ff98ecae30005afb7ab)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;该网络由三个部分组成。浅层特征提取、深层特征提取和高质量图像重建。其中，深层特征提取模块由一系列残差 Swin Transformer模块组成。Swin IR在 图像超分辨率、图像降噪 和JPEG 压缩图像增强三个任务取得了SOTA的成绩，因此选作本次比赛的baseline模型。

## 三、数据说明
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;该项目的数据来源于百度网盘AI大赛——图像处理挑战赛：文档图像超分的官方数据集。为了让网络更好地收敛，我们对数据集进行了数据增广、数据筛选等处理。

### 3.1 数据集介绍

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;在本次比赛最新发布的数据集中，所有的图像数据均由真实场景采集得到，再通过技术手段进行相应处理，生成可用的脱敏数据集。该任务为image-to-image的形式，因此源数据和对应的超分后数据均以图片的形式来提供。

**数据集构成**

训练数据集构成

```diff
|- train_data  
    |- x
    |- x2
    |- x4
 ```
测试数据集构成

```diff
|- test_data  
    |- x
 ```
* 本次比赛最新发布的数据集共包含训练集、A榜测试集、B榜测试集三个部分，其中训练集共3000个样本，A榜测试集共200个样本，B榜测试集共200个样本；
* x 为原始图像数据，x2为原始数据分辨率放大两倍后的数据，x4为原始数据分辨率放大四倍后的数据
**数据集可视化展示**

![](https://ai-studio-static-online.cdn.bcebos.com/feb65a7b18df4a86bd67f3b071073680c6dabab4eb6d4719bd68fb0027cb3377)


 

### 3.2 数据增广

* 由于Swin IR模型采用Transformer架构，因此对图像的尺寸较为敏感，过大的图像尺寸会导致显存溢出，过小的尺寸又会增加模型耗时，因此将模型裁剪为$128 \times 128$ 的patch作为网络输入。

* 为了提高网络鲁棒性，在数据集读取过程中，对图像块进行了随机翻转和旋转操作。

### 3.3 数据筛选

* 根据平均梯度的方法，筛选掉图像纹理较少或者空白的图像块。



## 四、代码实现

* **训练细节:** Swin IR 模型采用多阶段训练策略进行训练

* **与baseline差异：** 与已有baseline不同，这里我们对文档图像的2倍超分辨和4倍超分辨进行联合训练, 2倍超分辨和4倍超分辨损失按 0.3 和 0.7 的比例进行求和。

* **模型保存：** 网络每 500 次迭代保存一次临时模型,以便进行网络恢复; 每 5000 次迭代在测试集上进行一次测试, 选取测试分数最高的模型保存。



In [ ]:
# 解压数据集，虽然控制台可以解压，但是强烈建议在终端运行如下命令， patch_dataset_Step1.zip有近4万*3张图像块，输出过多
# 可能会导致控制台卡顿，整个解压过程耗时约半个小时左右
!unzip -d /home/aistudio/ /home/aistudio/data/data173301/patch_dataset_Step1.zip
!unzip -d /home/aistudio/ /home/aistudio/data/data173301/patch_dataset_Step2.zip
!unzip -d /home/aistudio/ /home/aistudio/data/data173301/patch_dataset_Step3.zip

Archive:  /home/aistudio/data/data173301/patch_dataset_Step1.zip
replace /home/aistudio/patch_dataset/x4/sp_train_0282_s023.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# 安装paddle_msssim包
!pip install paddle_msssim

In [15]:
# 启动第一阶段训练代码
%cd /home/aistudio/work/
# 创建输出文件夹，用于模型保存
!mkdir output sample
!python train_Step1.py

/home/aistudio/work
mkdir: 无法创建目录"output": 文件已存在
mkdir: 无法创建目录"sample": 文件已存在
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
train 38507 val 786
----------------------------------------------------------------------------------------------
     Layer (type)                 Input Shape               Output Shape         Param #    
       Conv2D-1               [[4, 3, 128, 128]]         [4, 180, 128, 128]       5,040     
      LayerNorm-1              [[4, 16384, 180]]          [4, 16384, 180]          360      
     PatchEmbed-1            [[4, 180, 128, 128]]         [4, 16384, 180]           0       
       Dropout-1               [[4, 16384, 180]]          [4, 16384, 180]           0       
      LayerNorm-2              [[4, 16384, 180]]          [4, 16384, 180]          360      
       Linear-1    

In [13]:
# 启动第二阶段训练代码,此处需要加载Step1的最佳模型进行权重初始化
%cd /home/aistudio/work/
!python train_Step2.py

/home/aistudio/work
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
train 66224 val 3486
Resume model from model_best_Step1.param


In [ ]:
# 启动第三阶段训练代码,此处需要加载Step2的最佳模型进行权重初始化
%cd /home/aistudio/work/
!python train_Step3.py

## 五、效果展示

* 可视化结果保存于/home/aistudio/result/x2和/home/aistudio/result/x4文件夹。由于生成的超分图像过大,请下载到本地查看。

* **提交：** 可直接下载/home/aistudio/submit_project.zip文件或自行打包/home/aistudio/predict_code内的脚本和模型进行提交。

In [ ]:
%cd  /home/aistudio
!mkdir /home/aistudio/result/ /home/aistudio/result/x/  /home/aistudio/result/x2 /home/aistudio/result/x4
!cp /home/aistudio/work/best_model/model_Step3_515k.param /home/aistudio/predict_code
%cd /home/aistudio/predict_code/
!python predict.py /home/aistudio/result/x/  /home/aistudio/result/x2   /home/aistudio/result/x4

## 六、总结和可以改进的点

* 目前Swin IR采用128×128 的图像块，batch size = 4 作为输入，耗时为 7.6s。 如果采用256×256的图像块或更大尺寸作为输入耗时可能降到5s以内; 因此可以先在128×128块图像块上进行预训练,再在256×256图像块上进行微调。

* 对于纹理较少图像块可以考虑采用cubic插值法,其他部分采用Swin IR进行超分辨率 来缩短时间。


```

```